# Small moldic pore type identification using Encoder Neural Networks

In this notebook we will focus on the classification of only small moldic pores in the image.
This is relevant because small moldic pores are quite different in shape and its surroundings when compared to large moldic pores.

In [ ]:
import os
print(os.getcwd())

In [ ]:
from pre_sal_ii.improc import colorspace

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Extracting pores from image

In [ ]:
def scale_image_and_save(input_file, output_path, scale_percent):
    image_name = os.path.splitext(os.path.basename(input_file))[0]
    output_file = f"{output_path}/{image_name}_{scale_percent}.jpg"

    if os.path.exists(output_file):
        return

    # Reading an image in default mode:
    inputImage = cv2.imread(input_file)

    # Set the scaling factors
    scale_percent = 25  # e.g., downscale to 25%

    # Calculate the new dimensions
    width = int(inputImage.shape[1] * scale_percent / 100)
    height = int(inputImage.shape[0] * scale_percent / 100)
    new_dimensions = (width, height)

    resized_image = cv2.resize(inputImage, new_dimensions, interpolation=cv2.INTER_AREA)
    os.makedirs(f"{output_path}/", exist_ok=True)
    cv2.imwrite(
        output_file,
        resized_image,
        [cv2.IMWRITE_JPEG_QUALITY, 99]
        )

In [ ]:
image_name = "ML-tste_original"
path = f"../data/classificada_01/{image_name}.jpg"
scale_image_and_save(path, "../out/classificada_01/", 25)

image_name = "ML-tste_classidicada"
path = f"../data/classificada_01/{image_name}.jpg"
scale_image_and_save(path, "../out/classificada_01/", 25)

In [ ]:
image_name = "ML-tste_original"
path = f"../out/classificada_01/{image_name}_25.jpg"
inputImage = cv2.imread(path)
plt.imshow(inputImage[:,:,::-1])

# BGR to CMKY:
inputImageCMYK = colorspace.bgr2cmyk(inputImage)

In [ ]:
(C, M, Y, K) = (inputImageCMYK[..., 0],
                inputImageCMYK[..., 1],
                inputImageCMYK[..., 2],
                inputImageCMYK[..., 3])

In [ ]:
binaryImage = cv2.inRange(
    inputImageCMYK,
    (92,   0,   0,   0),
    (255, 255,  64, 196))
binaryImage

In [ ]:
plt.imshow(binaryImage, cmap='gray')
cv2.imwrite("../out/some_prefilter.jpg", binaryImage)


In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
binaryImage = cv2.morphologyEx(binaryImage, cv2.MORPH_ERODE, kernel, iterations=1)
binaryImage = cv2.morphologyEx(binaryImage, cv2.MORPH_DILATE, kernel, iterations=1)
binaryImage = cv2.morphologyEx(binaryImage, cv2.MORPH_DILATE, kernel, iterations=1)
binaryImage = cv2.morphologyEx(binaryImage, cv2.MORPH_ERODE, kernel, iterations=1)

plt.imshow(binaryImage, cmap='gray')
cv2.imwrite("../out/some.jpg", binaryImage)
porosidade = np.sum(binaryImage/255)/binaryImage.size
print(f"porosidade = {porosidade}")

In [ ]:
from skimage.measure import label, regionprops

label_img = label(binaryImage)
regions = regionprops(label_img)

In [ ]:
all_objs = []
for it, region in enumerate(regions):
    ys = (region.coords.T[0] - label_img.shape[0]/2)/(label_img.shape[0]/2)
    xs = (region.coords.T[1] - label_img.shape[1]/2)/(label_img.shape[1]/2)
    obj = {
        "area": region.area,
        "max-dist": max((ys**2 + xs**2)**0.5),
    }
    all_objs.append(obj)

df = pd.DataFrame(all_objs)

In [ ]:
max_dist = max(df["max-dist"])
colored_image3 = np.zeros((*label_img.shape, 3), dtype=np.uint8)
for it, region in enumerate(regions):
    if df["max-dist"].iloc[it] <= max_dist*0.8:
        color_value = np.random.randint(0, 255, size=3)
        colored_image3[region.coords.T[0], region.coords.T[1]] = color_value

In [ ]:
plt.imshow(colored_image3)
cv2.imwrite("../out/colored_regions_rem_dist.jpg", colored_image3[:,:,::-1])


In [ ]:
ax = df.area.hist(bins=1000)
ax.set_xlim([0, 25000])

In [ ]:
max_dist = max(df["max-dist"])
colored_image3 = np.zeros((*label_img.shape, 3), dtype=np.uint8)
for it, region in enumerate(regions):
    if df["max-dist"].iloc[it] <= max_dist*0.8 and df["area"].iloc[it] <= 8000:
        color_value = np.random.randint(0, 255, size=3)
        colored_image3[region.coords.T[0], region.coords.T[1]] = color_value

In [ ]:
plt.imshow(colored_image3)


In [ ]:
max_dist = max(df["max-dist"])
pores_image3 = np.zeros(label_img.shape, dtype=np.uint8)
for it, region in enumerate(regions):
    if df["max-dist"].iloc[it] <= max_dist*0.8 and df["area"].iloc[it] <= 8000:
        color_value = 255
        pores_image3[region.coords.T[0], region.coords.T[1]] = color_value

In [ ]:
print(pores_image3.shape)
plt.imshow(pores_image3, cmap="gray")


## Loading categorized image

In [ ]:
image_name = "ML-tste_classidicada"
path = f"../out/classificada_01/{image_name}_25.jpg"
inputImage_cl = cv2.imread(path)
plt.imshow(inputImage_cl[:,:,::-1])


In [ ]:
binaryImage_clRed = cv2.inRange(
    inputImage_cl,
    #  B,   G,   R
    (  0,   0, 240),
    (  5,   5, 255))
plt.imshow(binaryImage_clRed, cmap='gray')

In [ ]:
cv2.imwrite("../out/binaryImage_clRed.jpg", binaryImage_clRed)


In [ ]:
import torch

image_pred_true = np.zeros([*binaryImage_clRed.shape, 3], dtype=np.uint8)
image_pred_true = torch.tensor(image_pred_true, dtype=torch.uint8).permute(2, 0, 1)
image_pred_true[2,:,:] = torch.tensor(binaryImage_clRed, dtype=torch.uint8)
image_pred_true[0,:,:] = torch.tensor(pores_image3, dtype=torch.uint8)
image_pred_true = image_pred_true.permute(1, 2, 0)
image_pred_true = image_pred_true.numpy()
plt.imshow(image_pred_true[:,:,::-1])


In [ ]:
image_pred_true

In [ ]:
from skimage.measure import label, regionprops

label_img = label(binaryImage_clRed)
regions = regionprops(label_img)

In [ ]:
all_objs = []
for it, region in enumerate(regions):
    ys = (region.coords.T[0] - label_img.shape[0]/2)/(label_img.shape[0]/2)
    xs = (region.coords.T[1] - label_img.shape[1]/2)/(label_img.shape[1]/2)
    obj = {
        "area": region.area,
        "max-dist": max((ys**2 + xs**2)**0.5),
    }
    all_objs.append(obj)

df = pd.DataFrame(all_objs)

In [ ]:
df

In [ ]:
max_dist = max(df["max-dist"])
pores_image_clRed_2 = np.zeros(label_img.shape, dtype=np.uint8)
for it, region in enumerate(regions):
    if df["max-dist"].iloc[it] <= max_dist*0.8 and df["area"].iloc[it] <= 8000:
        color_value = 255
        pores_image_clRed_2[region.coords.T[0], region.coords.T[1]] = color_value

In [ ]:
print(pores_image_clRed_2.shape)
plt.imshow(pores_image_clRed_2, cmap="gray")


## Extracting features and targets

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
from pre_sal_ii.models.WhitePixelRegionDataset import WhitePixelRegionDataset

In [ ]:
from torch.utils.data import DataLoader
num_samples = 10000
dataset = WhitePixelRegionDataset(
    pores_image3, inputImage, binaryImage_clRed, num_samples=num_samples)
data_loader = DataLoader(dataset, batch_size=32, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
dataiter = iter(data_loader)
data = next(dataiter)
(img, imgTarget, centerPixel) = data
print(data)
print(len(data))
print(torch.min(img), torch.max(img))

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
for data in data_loader:
    for it, (img, imgTarget) in enumerate(zip(data[0], data[1])):
        if it >= 10: break
        img = img/255
        imgTarget = imgTarget/255
        # print(img.numpy().shape, img.dtype)
        axes[it//5*2+0, it%5].imshow(img.numpy()[:,:,::-1])
        axes[it//5*2+1, it%5].imshow(imgTarget.numpy(), cmap="gray", vmin=0, vmax=1)
    break

In [ ]:
import torch.optim as optim
from pre_sal_ii.models.EncoderNN import EncoderNN

model = EncoderNN().to(device)
# criterion = nn.MSELoss()
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(),
                        lr=1e-4,
                        # weight_decay=1e-3,
                       )
model

In [ ]:
best_model_state = None
best_model_loss = 0

In [ ]:
import copy
from tqdm import tqdm
import torch.nn.functional as F

num_epochs = 100
bar = tqdm(total=num_epochs*num_samples)
for epoch in range(num_epochs):
    model.train()
    for data in data_loader:
        imgs = data[0].to(device)
        imgs = imgs.permute(0, 3, 1, 2)
        # print(f"imgs.shape={imgs.shape}")
        imgs = imgs/255
        imgs = F.interpolate(
            imgs, size=(32, 32), mode='bilinear',
            align_corners=False)
        imgs = imgs.reshape(-1, 3*32*32)
        # print(f"imgs.shape={imgs.shape}")
        # break
        outputs = model(imgs)
        expected = data[1].to(device)/255
        expected = torch.squeeze(expected, 2)
        # print(expected)
        # print(f"outputs.shape={outputs.shape}")
        # print(f"expected.shape={expected.shape}")
        # break
        loss = criterion(outputs, expected)
        
        if best_model_state is None or loss.item() < best_model_loss:
            best_model_state = copy.deepcopy(model.state_dict())
            best_model_loss = loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        bar.update(32)

    print(f"epoch={epoch}, loss={loss.item():.4f}")

# model = SimpleCNN().to(device)
# model.load_state_dict(best_model_state)
# model.eval()

In [ ]:
torch.save(best_model_state, "../models/supervised-2-small-pores.bin")


In [ ]:
model2 = EncoderNN().to(device)
model2.load_state_dict(best_model_state)
model2.eval()

## Prediction

In [ ]:
dataset2 = WhitePixelRegionDataset(
    pores_image3, inputImage, binaryImage_clRed, num_samples=-1, seed=None)

In [ ]:
pred_image = np.zeros(binaryImage_clRed.shape, dtype=np.uint8)

count_gt_half = 0

with torch.no_grad():
    for it, (imgX, _, coords) in enumerate(tqdm(dataset2)):
        # print(f"coords.shape={coords.shape}")
        imgX = imgX.to(device)
        imgX = imgX.unsqueeze(0)
        imgX = imgX.permute(0, 3, 1, 2)
        # print(f"imgX.shape={imgX.shape}")
        imgX = imgX/255
        imgX = F.interpolate(
            imgX, size=(32, 32), mode='bilinear',
            align_corners=False)
        imgX = imgX.reshape(-1, 3*32*32)
        # print(f"imgX.shape={imgX.shape}")
        # break
        Y = model(imgX)

        pred_image[int(coords[0]), int(coords[1])] = float(Y[0,0])*255

        # if float(Y[0,0]) > 0.5:
        #     count_gt_half += 1
        #     print(f"{[coords[0], coords[1]]} -> {pred_image[coords[0], coords[1]]} (Y[0,0]={Y[0,0]})")
            
        # if it > 1000: break

In [ ]:
plt.imshow(pred_image, vmin=0, vmax=255, cmap="gray")
cv2.imwrite("../out/sup_pred_2_small_pores_model.jpg", pred_image)


In [ ]:
image_pred_true = np.zeros([*binaryImage_clRed.shape, 3], dtype=np.uint8)
image_pred_true = torch.tensor(image_pred_true, dtype=torch.uint8).permute(2, 0, 1)
image_pred_true[0,:,:] = torch.tensor(pores_image3, dtype=torch.uint8)
image_pred_true[1,:,:] = torch.tensor(pores_image_clRed_2, dtype=torch.uint8)
image_pred_true[2,:,:] = torch.tensor(pred_image, dtype=torch.uint8)
image_pred_true = image_pred_true.permute(1, 2, 0)
image_pred_true = image_pred_true.numpy()
plt.imshow(image_pred_true[:,:,::-1])
cv2.imwrite("../out/image_pred_true_small_pores_model.jpg", image_pred_true)
